# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>

# <h1><center>Actividad 04: Obteniendo Datos de la Web</center></h1>

Esta actividad busca aplicar conocimientos sobre lectura de datos desde la web en distintos formatos (scrapping y APIs) para la creación de un dataset unificado.

## Instrucciones

Este Notebook contiene las instrucciones a realizar para la actividad. 

<b>Al finalizarla, deben subir el Notebook y los archivos generados en un único archivo .zip, al módulo de la Actividad 04 en Canvas. Entregas posteriores al cierre de la actividad serán evaluadas con nota 1.0.</b>

## Actividad

Para esta actividad, queremos analizar la calidad del aire de las ciudades más pobladas del mundo. Para esto, realice los siguientes pasos:

**1. Extraer datos con web scrapping y API**

Vamos a extraer una lista de las ciudades más pobladas desde Wikipedia, específicamente en el siguiente URL:

`URL_1 = https://en.wikipedia.org/wiki/List_of_largest_cities#List`

Por otra parte, usaremos la [Open-Meteo API](https://open-meteo.com/). Este es un servicio open-source de meteorología que nos permitirá obtener datos como las coordenadas de una ciudad y sus parámetros climáticos, como la calidad del aire.

* 1.1 Utilizando las librerías `requests`y `BeautifulSoup`, obtenga todas las filas y columnas de la tabla de Wikipedia de las ciudades más grandes del mundo y genere un DataFrame a partir de ellas. Su DataFrame debe contener como mínimo las siguientes columnas: ciudad, país y población estimada.

* 1.2 Transforme la columna de población en valores numéricos y sólo deje las 20 mayores ciudades.

**2. Llamada a la API**

* 2.1 Ahora, utilizando `requests`, haga un llamado al siguiente URL de la API de Open-Meteo, reemplazando el valor `CIUDAD` con cada uno de los nombres de las ciudades de su DataFrame:

`URL_2 = https://geocoding-api.open-meteo.com/v1/search?name={CIUDAD}&count=1&language=en&format=json`

Haga una copia de su DataFrame anterior. En esta copia, agregue dos columnas nuevas y guarde los valores obtenidos de latitud y longitud (sin modificar el DataFrame original).

* 2.2 Con los datos de las coordenadas, podemos acceder a información sobre la calidad del aire actual disponible con Open-Meteo. Nuevamente, para todas las ciudades, utilice el URL dado para obtener el índices de calidad del aire (usaremos el europeo) y la cantidad de partículas en suspensión.

`URL_3 = https://air-quality-api.open-meteo.com/v1/air-quality?latitude={LAT}&longitude={LON}&current=european_aqi,pm10,pm2_5`

Guarde los valores obtenidos en nuevas columnas del mismo DataFrame.

* 2.3 En la documentación de Open-Meteo ([aquí](https://open-meteo.com/en/docs/air-quality-api)), podemos ver el significado de los valores del índice European AQI. Utilizando la función `aqi2str()` entregada, genere una nueva columna `Air Quality` (string) a partir de los valores que obtuvo mediante la API.

* 2.4 Revise los valores obtenidos. ¿Tienen sentido? Si hay valores que considere inválidos o "outliers" (extremadamente altos), descártelos del dataset.

**3. Visualización**

Vamos a generar dos visualizaciones a partir de las ciudades con las que hemos trabajado. Para esto, usaremos una nueva librería de visualización llamada `plotly.express`. Plotly permite generar gráficos interactivos, con tooltips donde podemos mostrar información adicional de nuestro DataFrame, lo cual los hace muy convenientes para la exploración de un dataset.

Lea y complete el código entregado con los valores de su DataFrame. Ejecute las celdas y responda:

* Entre las ciudades más pobladas, ¿cuál es la calidad de aire más común?

* ¿Cómo es la relación entre tamaño de población y calidad del aire de las ciudades?

* ¿Hay algún lugar del mundo donde se vea una mayor concentración de grandes ciudades? Si la hay, ¿cómo es la calidad del aire en estas zonas?

## Rúbrica

- Si han hecho todo y sólo hay errores menores: 7.0
- Si sólo llegaron hasta la parte 2.1: 5.0
- Menos que eso: 1.0

### 0. Algunas librerías

Las siguientes son algunas de las librerías que recomendamos usar para esta Actividad. Puede agregar más si lo requiere.

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

### 1. Extraer datos

#### 1.1 Respuesta:

In [66]:
url_cities = "https://en.wikipedia.org/wiki/List_of_largest_cities"
# Codigo clase
headers = {"User-Agent": "imt2200-Act-4"}
page = requests.get(url_cities, headers=headers, timeout=10)
cities_soup = bs(page.text)
tablas = cities_soup.find_all('table')
tabla_cities = tablas[1]
df_cities = pd.DataFrame(columns = ['city','country','population'])
tabla_cities



<table class="static-row-numbers plainrowheaders vertical-align-top sticky-header-multi sort-under sortable wikitable">
<tbody><tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th rowspan="2" scope="col">City<sup class="reference" id="cite_ref-12"><a href="#cite_note-12"><span class="cite-bracket">[</span>a<span class="cite-bracket">]</span></a></sup>
</th>
<th rowspan="2" scope="col">Country
</th>
<th rowspan="2" scope="col" style="max-width:4em">UN 2018 population estimates<sup class="reference" id="cite_ref-13"><a href="#cite_note-13"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th colspan="4" scope="col"><a href="/wiki/City_proper" title="City proper">City proper</a><sup class="reference" id="cite_ref-14"><a href="#cite_note-14"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th colspan="3" scope="col"><a href="/wiki/Urban_area" title="Urban area">Urban area</a><sup c

In [95]:

for row in tabla_cities.find_all('tr')[:21]:
    cols = row.find_all("th")
    colas = row.find_all("td")
    if len(colas) > 0 and len(cols) > 0:
        colas = [col.text.strip() for col in colas]
        cols = [col.text.strip() for col in cols]
        ctiy = cols[0]
        country = colas[0]
        population = colas[1]
        new_row = pd.DataFrame({'city': ctiy, 'country': country, 'population': population}, index=['city'])
        df_cities = pd.concat([df_cities, new_row], ignore_index=True)

df_cities





,city,country,population
0,Tokyo,NaN,NaN
1,Delhi,NaN,NaN
2,Shanghai,NaN,NaN
3,São Paulo,NaN,NaN
4,Mexico City,NaN,NaN
...,...,...,...
414,Buenos Aires,Argentina,"14,967,000"
415,Chongqing,China,"14,838,000"
416,Istanbul,Turkey,"14,751,000"
417,Manila,Philippines,"13,482,000"


#### 1.2 Respuesta:

In [ ]:
URL_2 = f"https://geocoding-api.open-meteo.com/v1/search?name={CIUDAD}&count=1&language=en&format=json"


### 2. Uso de API

#### 2.1 Respuesta:

In [ ]:
# 2.1

#### 2.2 Respuesta:

In [ ]:
# 2.2

#### 2.3 Respuesta:

In [ ]:
# ==== CODIGO ENTREGADO - NO MODIFICAR ====
air_quality = {
    "Good": [0, 20],
    "Fair": [20, 40],
    "Moderate": [40, 60],
    "Poor": [60, 80],
    "Very Poor": [80, 100],
    "Extremely Poor": [100, float('inf')]
}

def aqi2str(aqi):
    for key, (low, high) in air_quality.items():
        if low <= aqi < high:
            return key
    return "Unknown"

In [ ]:
# 2.3

#### 2.4 Respuesta:

In [ ]:
# 2.4

### 3. Visualizar datos

* ¿Cómo son los valores de calidad de aire para las ciudades más pobladas? ¿Cuál es lo más común?

* ¿Cómo es la relación entre tamaño de población y calidad del aire de una ciudad?

* ¿Hay algún lugar del mundo donde se vea una mayor concentración de grandes ciudades? Si la hay, ¿cómo es la calidad del aire?

In [ ]:
# Figura 1: Barplot de calidad del aire
import plotly.express as px

by_quality = new_df.groupby('Air Quality').size().reset_index(name='Count')

fig = px.bar(by_quality,
            x='Air Quality',
            y='Count',
            title="Calidad del aire de 80 ciudades más pobladas del mundo",
            labels={
                "Count": "Cantidad de ciudades",
                "Air Quality": "Calidad del aire"
            },
            color='Air Quality')

fig.update_layout(
    height=400,
    width=900,
)
fig.update_xaxes(categoryorder='array',
                 categoryarray= ["Good", "Fair", "Moderate", "Poor", "Very Poor", "Extremely Poor"]
)
fig.show()

#### Respuesta:

In [ ]:
# Figura 2: Scattermap entre población y calidad del aire

fig = px.scatter(df_filtered,
                 x="Population",
                 y="eu_aqi",
                 title="Relación entre población y calidad del aire",
                 labels={
                     "Population": "Población",
                     "eu_aqi": "Calidad del aire (EU AQI)"
                    },
                    hover_data={
                        "City": True,
                        "Country": True,
                        "Population": True,
                        "eu_aqi": True,
                        "Air Quality": True
                        }
                )

fig.update_layout(
    height=500,
    width=900,
)

#### Respuesta:

In [ ]:
# Figura 3: Mapa mundial de ciudades más pobladas

fig = px.scatter_geo(data_frame=df_filtered, # Su dataframe
                    lat='lat', # Columna de latitud
                    lon='lon', # Columna de longitud
                    color='eu_aqi', # Columna que representa el color de los puntos
                    hover_name='City', # Columna para el titulo del tooltip
                    projection="natural earth",
                    color_continuous_scale=px.colors.sequential.Inferno_r,
                    title="Calidad del aire de ciudades más pobladas", # Titulo del grafico
                    hover_data={
                        # Qué columnas mostrar en el tooltip
                        "Country": True,
                        "eu_aqi": True,
                        "Air Quality": True
                        # Puede agregar otras...
                    },
                )

fig.update_layout(
    margin={"r":0,"t":50,"l":0,"b":0}, # Márgenes del gráfico
    height=600, # Altura del gráfico
    width=800, # Ancho del gráfico
)
fig.update_traces(
    marker=dict(size=10), # Tamaño de los puntos
)
fig.show()

#### Respuesta: